## An EDA of Bank Customer Transactions by their age and location demographic in the year 2016
Time frame: 1st August, 2016 -> 21st October, 2016

In [3]:
import pandas as pd
from sqlalchemy import create_engine

db_username = 'root'
db_password = '*********'
db_host = 'localhost'
db_name = 'Banking'

connection_string = f"mysql+mysqlconnector://{db_username}:{db_password}@{db_host}/{db_name}"
engine = create_engine(connection_string)

query = "SELECT * FROM bank_customer_transactions_dedup;"

try:
    print("Importing from MySQL database")
    df = pd.read_sql(query, engine)
    print("Data import done\n")
finally:
    engine.dispose()
    print("\nDatabase connection closed.")

Importing from MySQL database
Data import done


Database connection closed.


In [14]:
df

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5
...,...,...,...,...,...,...,...,...,...
1048562,T1048563,C8020229,8/4/90,M,NEW DELHI,7635.19,18/9/16,184824,799.0
1048563,T1048564,C6459278,20/2/92,M,NASHIK,27311.42,18/9/16,183734,460.0
1048564,T1048565,C6412354,18/5/89,M,HYDERABAD,221757.06,18/9/16,183313,770.0
1048565,T1048566,C6420483,30/8/78,M,VISAKHAPATNAM,10117.87,18/9/16,184706,1000.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048567 entries, 0 to 1048566
Data columns (total 9 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   TransactionID            1048567 non-null  object 
 1   CustomerID               1048567 non-null  object 
 2   CustomerDOB              1045170 non-null  object 
 3   CustGender               1047467 non-null  object 
 4   CustLocation             1048416 non-null  object 
 5   CustAccountBalance       1046198 non-null  float64
 6   TransactionDate          1048567 non-null  object 
 7   TransactionTime          1048567 non-null  int64  
 8   TransactionAmount (INR)  1048567 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 72.0+ MB


In [6]:
## Prelim cleaning (Converting Time column to HH:MM:SS format)

df['TransactionTime'] = df['TransactionTime'].apply(lambda x: f"{int(x):06d}")
df['TransactionTime'] = pd.to_datetime(df['TransactionTime'], format="%H%M%S").dt.time

In [104]:
## Converting date columns from string to date format (dd/mm/yy)

## merged_df['CustomerDOB'] = pd.to_datetime(merged_df['CustomerDOB'], format="%d/%m/%y")
## merged_df['TransactionDate'] = pd.to_datetime(merged_df['TransactionDate'], format="%d/%m/%y")

## Getting error here, so I will check further

In [7]:
invalid_dobs = df[~df['CustomerDOB'].astype(str).str.match(r'^\d{1,2}/\d{1,2}/\d{2}$', na=False)]
print("Invalid DOB entries:", len(invalid_dobs))
print(invalid_dobs['CustomerDOB'].unique())
invalid_tx_dates = df[~df['TransactionDate'].astype(str).str.match(r'^\d{1,2}/\d{1,2}/\d{2}$', na=False)]
print("Invalid TransactionDate entries:", len(invalid_tx_dates))
print(invalid_tx_dates['TransactionDate'].unique())

Invalid DOB entries: 60736
['1/1/1800' nan]
Invalid TransactionDate entries: 0
[]


In [8]:
## Since pandas auto formats years before '66 as 2066, this will lead to age errors
# Will need to manually preprocess to avoid this error here

def fix_year(date_str):
    if pd.isna(date_str):
        return pd.NaT
    try:
        parts = date_str.split('/')
        if len(parts) == 3:
            day, month, year = parts
            year = int(year)
            if year < 10:    ## Assuming no one born before 1910 will be making transactions in 2016
                year += 2000
            else:
                year += 1900
            return f"{day}/{month}/{year}"
        else:
            return pd.NaT
    except:
        return pd.NaT

## Applying this fix before parsing
df['CustomerDOB'] = df['CustomerDOB'].replace('1/1/1800', pd.NaT)
df['CustomerDOB'] = df['CustomerDOB'].apply(fix_year)
df['CustomerDOB'] = pd.to_datetime(df['CustomerDOB'], format="%d/%m/%Y", errors='coerce')
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'], format="%d/%m/%y")

In [9]:
## Age column for future analyses
transaction_date = pd.to_datetime('2016-10-18') ## Assuming the year of the transaction date as the current year
df['Age'] = (transaction_date - df['CustomerDOB']).dt.days // 365

In [10]:
print(df['Age'].describe()) ##Checking for unnatural values in Age column

count    987831.000000
mean         30.876751
std           8.805353
min           7.000000
25%          25.000000
50%          29.000000
75%          34.000000
max         106.000000
Name: Age, dtype: float64


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048567 entries, 0 to 1048566
Data columns (total 10 columns):
 #   Column                   Non-Null Count    Dtype         
---  ------                   --------------    -----         
 0   TransactionID            1048567 non-null  object        
 1   CustomerID               1048567 non-null  object        
 2   CustomerDOB              987831 non-null   datetime64[ns]
 3   CustGender               1047467 non-null  object        
 4   CustLocation             1048416 non-null  object        
 5   CustAccountBalance       1046198 non-null  float64       
 6   TransactionDate          1048567 non-null  datetime64[ns]
 7   TransactionTime          1048567 non-null  object        
 8   TransactionAmount (INR)  1048567 non-null  float64       
 9   Age                      987831 non-null   float64       
dtypes: datetime64[ns](2), float64(3), object(5)
memory usage: 80.0+ MB


In [12]:
print("Missing values:\n", df.isnull().sum())
print("Duplicate rows:", df.duplicated().sum())

Missing values:
 TransactionID                  0
CustomerID                     0
CustomerDOB                60736
CustGender                  1100
CustLocation                 151
CustAccountBalance          2369
TransactionDate                0
TransactionTime                0
TransactionAmount (INR)        0
Age                        60736
dtype: int64
Duplicate rows: 0


In [13]:
df['CustGender'].value_counts()

CustGender
M    765530
F    281936
T         1
Name: count, dtype: int64

In [14]:
df['CustGender'] = df['CustGender'].fillna('Unknown')
df['CustLocation'] = df['CustLocation'].fillna('Unknown')

In [15]:
## Missing values for Account Balance will be replaced with median or mean (Depending on how skewed the data is)
df['CustAccountBalance'].describe()

count    1.046198e+06
mean     1.154035e+05
std      8.464854e+05
min      0.000000e+00
25%      4.721760e+03
50%      1.679218e+04
75%      5.765736e+04
max      1.150355e+08
Name: CustAccountBalance, dtype: float64

In [16]:
mean = df['CustAccountBalance'].mean()
median = df['CustAccountBalance'].median()

print(f"Mean: {mean:,.2f}")
print(f"Median: {median:,.2f}")
print("Skewed Right\nThere are a few very large account balances; Outliers inflating average"
      if mean > median else "Skewed Left" if mean < median else "Symmetric")

Mean: 115,403.54
Median: 16,792.18
Skewed Right
There are a few very large account balances; Outliers inflating average


In [17]:
# Checking skewness for account balances and transaction amounts
balance_skew = df['CustAccountBalance'].skew()

transaction_skew = df['TransactionAmount (INR)'].skew()

print("Account Balance Skewness:", balance_skew)
print("Transaction Amount Skewness:", transaction_skew)

if balance_skew>5 and transaction_skew>5:
    print("\nThe data is highly skewed and using median for computing averages is much safer for this dataset.")

Account Balance Skewness: 60.53033399781306
Transaction Amount Skewness: 47.337251797156924

The data is highly skewed and using median for computing averages is much safer for this dataset.


In [18]:
median_balance = df['CustAccountBalance'].median()
df['CustAccountBalance'] = df['CustAccountBalance'].fillna(median_balance)

In [19]:
print("Remaing missing values:\n",df.isnull().sum())

Remaing missing values:
 TransactionID                  0
CustomerID                     0
CustomerDOB                60736
CustGender                     0
CustLocation                   0
CustAccountBalance             0
TransactionDate                0
TransactionTime                0
TransactionAmount (INR)        0
Age                        60736
dtype: int64


In [20]:
## Since 60736 rows only constitutes 1% of the entire dataset, we can safely drop
# all such rows without worrying about health of the data
df = df.dropna(subset=['CustomerDOB'])
print("Remaing missing values:\n",df.isnull().sum())

Remaing missing values:
 TransactionID              0
CustomerID                 0
CustomerDOB                0
CustGender                 0
CustLocation               0
CustAccountBalance         0
TransactionDate            0
TransactionTime            0
TransactionAmount (INR)    0
Age                        0
dtype: int64


In [21]:
df['Age'] = df['Age'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 987831 entries, 0 to 1048566
Data columns (total 10 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   TransactionID            987831 non-null  object        
 1   CustomerID               987831 non-null  object        
 2   CustomerDOB              987831 non-null  datetime64[ns]
 3   CustGender               987831 non-null  object        
 4   CustLocation             987831 non-null  object        
 5   CustAccountBalance       987831 non-null  float64       
 6   TransactionDate          987831 non-null  datetime64[ns]
 7   TransactionTime          987831 non-null  object        
 8   TransactionAmount (INR)  987831 non-null  float64       
 9   Age                      987831 non-null  int64         
dtypes: datetime64[ns](2), float64(2), int64(1), object(5)
memory usage: 82.9+ MB


/var/folders/8g/00b3gbs92n16plbgtl71gh8h0000gn/T/ipykernel_56784/2019209778.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Age'] = df['Age'].astype(int)


In [22]:
# Quick Summary
print(df[['TransactionAmount (INR)', 'CustAccountBalance', 'TransactionDate']].describe())

       TransactionAmount (INR)  CustAccountBalance  \
count             9.878310e+05        9.878310e+05   
mean              1.453714e+03        1.058525e+05   
min               0.000000e+00        0.000000e+00   
25%               1.510000e+02        4.601910e+03   
50%               4.312100e+02        1.607897e+04   
75%               1.126000e+03        5.357282e+04   
max               1.560035e+06        1.150355e+08   
std               6.145956e+03        8.168791e+05   

                     TransactionDate  
count                         987831  
mean   2016-08-25 04:00:58.732313344  
min              2016-08-01 00:00:00  
25%              2016-08-12 00:00:00  
50%              2016-08-25 00:00:00  
75%              2016-09-06 00:00:00  
max              2016-10-21 00:00:00  
std                              NaN  


## LOCATION FILTER LOGIC
    Foreign customers will be dropped and exported out to another dataframe (For a future use case revolving foreign clients), as this will skew the data considerably more due to higher PPP in foreign countries.

In [23]:
tempdf = df.copy()

In [24]:
foreign = ['ABU DHABI', 'ACCRA', 'Addis Ababa', 'AGNES WATER', 'AJMAN', 'AL AIN', 'AL JUBAIL', 'AL NAHDA', 'AL NAKHEEL',
           'AL QUOZ IND', 'AL SALMIYA', 'AL WAKRA', 'ALAIN', 'ALBION', 'Amsterdam', 'AREZZO', 'ARUSHA', 'ASHFIELD', 'AUCKLAND',
           'AUSTIN', 'BAHRAIN', 'BAIRNSDALE', 'BANGKOK', 'BASINGSTOKE', 'BEDFORD', 'BEAVERTON', 'BERGEN', 'BIETIGHEIM',
           'BIETIGHEIM BISSINGEN', 'BIRMINGHAM', 'BORAS', 'BOX DUBAI DUBAI', 'BOX RIYADH', 'BRUSSEL', 'BRUXELLES', 'BUDAPEST', 
           'BURAIDAH', 'BUR DUBAI', 'BURWOOD EAST', 'CAMPBELL', 'CAMP HILL', 'CARINDALE', 'CASTLE HILL', 'CENTER DUBAI', 'CHARLOTTE',
           'CHESTER', 'CHICAGO', 'CHISOPANI', 'CLARKSBURG M MD', 'CLERMONT FERRAND', 'CLEAR WATER', 'COCKEYSVILLE', 'COPENHAGEN',
           'CORPUS CHRISTI', 'COVENTRY', 'CROYDON', 'CUMMING', 'CUPERTINO', 'CYPRESS', 'DAMMAM', 'DANDENONG', 'DAR ES SALAAM',
           'DARES SALAAM', 'DAYTON', 'DEN HAAG', 'DEVONSHIRE', 'DHAHRAN', 'DOCKLANDS', 'DOHA', 'DOYLESTOWN',
           'DUBAI UNITED ARAB EMIRATES DUBAI', 'DUBAI UAE', 'DUBLIN', 'EAGAN', 'EAST RIFFA', 'EDINBURGH', 'EDMONTON AB',
           'EDOGAWA KU', 'ELPHINSTONE', 'ERLANGEN', 'ETOBICOKE', 'EXTON', 'FAHAHEL', 'FARMINGTON HILLS', 'FLORIDA',
           'FONTENAY AUX ROSES', 'FRAMINGHAM', 'FREMONT', 'FUJAIRAH', 'FZCO DUBAI', 'GAITHERSBURG', 'GLASGOW', 'GLEN HUNTLY',
           'GLENGOWRIE', 'GOVERNORATE MANAMA', 'GUILDERLAND', 'HAARLEM', 'HAMBURG', 'HAMPSHIRE', 'HAMPTON PARK', 'HATFIELD',
           'HAYES', 'HEIDENHEIM', 'HELSINKI', 'HENDERSON', 'HERNDON', 'HOFFMAN ESTATES', 'HOF UF', 'HONGKONG', 'HOOFDDORP',
           'HOUSTON', 'ILFORD', 'IPOH', 'IRVINE', 'ISLAMABAD', 'JABAL ALI', 'JEDDAH', 'JEBEL ALI', 'JERSEY CITY', 'Johannesburg',
           'JUBAIL', 'JUBAILI IND CITY', 'JUFFAIR', 'KANATA', 'KAYLA', 'KHAMIS MUSHYAT', 'KHOBAR', 'KIGALI', 'KILLORGLIN',
           'KUALA BELAIT', 'KUALA LUMPUR', 'KUALALUMPUR', 'KUWAIT', 'KUWAIT CITY', 'KYOTO', 'LEEDS', 'LODZ', 'LUTON', 'MAARSSEN',
           'MAHBOULA', 'MALVERN', 'MANAMA', 'MANGAF', 'MANNHEIM', 'MARSEILLE', 'MCKINNEY', 'MELBOURNE', 'MEMPHIS', 'MISSISSAUGA',
           'MOBARA', 'MOSCOW', 'MOUNT HAWTHORN', 'MOUNTAIN VIEW', 'MUNICH', 'MUSCAT', 'NAIROBI', 'NAWALPARASI', 'NEGARA',
           'NEW MILFORD', 'NEW TAIPEI CITY', 'NEW YORK', 'NORBURY', 'NORTH CAROLINA', 'NORTH SYDNEY', 'NORWICH', 'NOTTINGHAM',
           'NSW', 'OKHALDHUNGA', 'OLDHAM', 'OMAN', 'OMAN MUSCAT', 'ORLANDO', 'ORPINGTON', 'OSAKA', 'OTTERFING', 'OUTRAM', 'PALPA',
           'PARRAMATTA', 'PEORIA', 'PERTH', 'PETALING JAYA', 'PHOENIX', 'PORT ELIZABETH', 'QUAKERS HILL', 'QUINCY', 'RAS AL KHAIMAH',
           'RAS TANURA', 'READING', 'Congo', 'RIFFA', 'RIIHIMAKI', 'RIVERSIDE', 'ROME', 'RUWI', 'SAFAT', 'SALALAH', 'SALMIYA', 'SALWA',
           'SANDHAUSEN', 'SAN JOSE', 'SANTA CLARA', 'SANTIAGO', 'SAUDI ARABIA', 'SCARBOROUGH', 'SEATTLE', 'SELANGOR', 'SENAI',
           'SHARJAH', 'SHARJAH KARAMA', 'SHARJAH MAIN CITY', 'SHEIKH ZAYED', 'SHEBOYGAN', 'SILVER SPRING', 'SKIEN', 'SOLIHULL',
           'SOMALI REGION', 'SOUTH HEDLAND', 'SOUTH MORANG', 'SOUTH WINDSOR', 'SPEYER', 'SPRING HILL', 'STAFFORD', 'STAINES',
           'STILLORGAN', 'STOCKHOLM', 'STOCKPORT', 'SULTANATE OF OMAN', 'SUNNYVALE', 'SURREY', 'SWEILEH', 'SWINDON', 'SYDNEY', 
           'TAINAN', 'TAMPINES', 'TAMPA', 'TOKYO', 'TOONGABBIE', 'UAE', 'ULSAN', 'UMM AL QUWAIN', 'UNION CITY', 'UXBRIDGE',
           'Virginia, USA', 'WALLISELLEN', 'WALSALL', 'WALTHAM', 'WATERLOO', 'WESTLANDS', 'WILLOUGHBY', 'WIRRAL', 'WOODSIDE',
           'YANBU', 'YANBU AL BAHAR', 'YOKOHAMA SHI', 'YORK']


foreign_locations = [
    'Roomford', 'Milton Keynes', 'Naperville', 'West Hartford', 'Port Hartcourt', 'Southall', 'Al Quoz IND 3',
    'Abu Dhabi 303', 'AL JUBAIL INSUDTRIAL', 'PO BOX 28483 DUBAI', 'RENTON', 'SULTANATE OF OMAN', 'AHMADI', 'Mombasa',
    'Sakaka', 'Brooklyn', 'Lagos', 'Liverpool', 'Brampton', 'Edison', 'Jannusan', 'Jizan', 'Mesaieed', 'Antwerpen',
    'Hung Hom', 'Leiden','Irving', 'Dubai', 'Wrzesnia', 'Al Khobar', 'London', 'Raleigh', 'Singapore', 'Kuala Lampur',
    'Abu Dhabi', 'Box Dubai', 'Shah Alam', 'Riyadh', 'Munchen', 'SULTANATE OFOMAN', 'FARWANIYA', 'HOFUF','JOHANNESBERG',
    'ADDIS ABABD'
]
foreign_locations_upper = [loc.upper() for loc in foreign_locations]

total_foreign_list = list(set(foreign) | set(foreign_locations_upper))

In [25]:
import re
tempdf['city'] = tempdf['CustLocation'].astype(str).str.strip()

known_cities = list(dict.fromkeys([
    'Pune', 'Nagpur', 'Greater Noida', 'Bangalore', 'Hyderabad', 'Chennai', 'Kolkata',
    'Ahmedabad', 'Gurgaon', 'Navi Mumbai', 'K V Rangareddy', 'Thane', 'Faridabad',
    'Noida', 'Mumbai', 'New Delhi', 'Shillong', 'Bengaluru', 'Rewari', 'Ghinghartola',
    'Guntur', 'Roorkee', 'Burdwan', 'Raigarh', 'Bhilai', 'Kancheepuram', 'Jaipur',
    'Aurangabad', 'Katra', 'Khammam', 'Vijayawada', 'Ranchi', 'Abohar', 'Tirupati',
    'Kanyakumari', 'Delhi', 'Amritsar', 'Mohali', 'Kanpur', 'Bidar', 'Champaran', 'Jorhat', 'Tripura', 'Dimapur',
    'Miramar', 'Jalpaiguri', 'Gangtok', 'Patna', 'Jamshedpur', 'Agra', 'Port Blair', 'Dhanbad', 'Ghaziabad',
    'Mahendergarh', 'Ludhiana', 'Tirupur'
]))

# Words to remove entirely from CustLocation
remove_terms = ["DT", "DIST", "PO", "DISTRICT", "DISTT", "TALUK", "URBAN", "RURAL", "BUNGLOWS", "BUNGALOWS",
                "CANTT", "W", "E", "PURBI", "LIMITED", "SCHOOL", "VPO", "CENTRE", " I E","STORES","VILLAGE", "ROAD",
                "OFFICE", "RD", "HOSPITAL"]
pattern_terms = r'\b(?:' + '|'.join(remove_terms) + r')\b'

# Replacement map for known corrections (Checking these first to override other processes)
replace_map = {
    "PARGANAS": "KOLKATA", "PARAGANAS": "KOLKATA", "KANNIYAKUMARI": "KANYAKUMARI", "BANGALURU": "BANGALORE",
    "BANGALROE": "BANGALORE", "BANGLORE": "BANGALORE", "CHEENAI": "CHENNAI", "SAS NAGAR":"MOHALI", "BANGALORE":"BENGALURU",
    "CINEMA HAORA":"HOWRAH", "KHARGHAR":"NAVI MUMBAI", "RANGA REDDY":"KV RANGAREDDY", "RANGAREDDY":"KV RANGAREDDY",
    "TALEIGAD":"TALEIGAO", "RANGA REDDY SERILINGAMPALLY":"KV RANGAREDDY", "G B NAGAR":"GAUTAM BUDDHA NAGAR",
    "KAIRA":"KHEDA", "SECUDERABAD":"SECUNDERABAD", "PUDUCHRRY":"PUDUCHERRY", "CANACOWA":'CANACONA', "PANT NAGAR":"PANTNAGAR",
    "P G S":"KOLKATA", "GAUTAM BUDHNAGAR":"GAUTAM BUDDHA NAGAR", "PGNS NORTH":"KOLKATA", ".":"Unknown", "CALCUTTA":"KOLKATA",
    "RACHI":"RANCHI", "TIRUCHIRAPALLY":"TIRUCHIRAPALLI", "CHOOLAIMEDU":"CHENNAI", "BANGLORE":"BENGALURU", "CHROMPET KPM":"CHROMEPET",
    "JAMESHEDPUR":"JAMSHEDPUR", "THIROVANANTHA PURAM":"THIRUVANANTHAPURAM", "MORDABAD":"MORADABAD", "JAMSAEDPUR":"JAMSHEDPUR",
    "KARKARDUMA":"DELHI", "GAUTAM BUDDHA":"GAUTAM BUDDHA NAGAR", "SINDRI":"DHANBAD", "ANDUL":"HOWRAH", "BANTWAL TQ":"BANTWAL",
    "MUMABI":"MUMBAI", "NEAR SWIMMING POOL AREA":"Unknown", "BANASHANKARIND STAGE":"BENGALURU"
}

def clean_location(text):
    text = text.upper()

     # First: Panvel / Mira Road check
    if re.search(r'\bPANVEL\b', text) or re.search(r'\bDOMBIVALI\b', text) or re.search(r'\bDOMBIVLI\b', text):
        return "NAVI MUMBAI"
    elif re.search(r'\bMIRA\s+ROAD\b', text):
        return "THANE"
    elif re.search(r'\bPAREL\b', text) or re.search(r'\bPOWAI\b', text) or re.search(r'\bKURLA\b', text) or re.search(r'\bMULUND\b', text):
        return "MUMBAI"
    elif re.search(r'\bSECUNDRABAD\b', text):
        return "SECUNDERABAD"

    for wrong, correct in replace_map.items():
        if wrong in text:
            return correct

    text = re.sub(r'\(.*?\)', '', text)
    text = re.sub(pattern_terms, '', text)
    text = re.sub(r'\d+', '', text)
    text = text.replace('-', ' ').replace(':', ' ')
    text = re.sub(r'\s+', ' ', text).strip()
    return text

tempdf['cleaned_location'] = tempdf['CustLocation'].apply(clean_location)

def find_city(row):
    for city in known_cities:
        if city.lower() in row['cleaned_location'].lower():
            return city.upper()
    return row['cleaned_location']  # Keep cleaned string if no match

tempdf['city'] = tempdf.apply(find_city, axis=1)

# We will remove foreign locations from our dataframe, since the the scope of the project is to cater to Indian clients.
# However, these foreign locations have been exported out to another dataframe for a future use case with foreign clients.
foreign_mask = tempdf['city'].str.upper().isin(total_foreign_list)
foreign_df = tempdf[foreign_mask].copy()
tempdf = tempdf[~foreign_mask].copy()

In [26]:
tempdf.drop('cleaned_location', axis=1, inplace=True)

In [ ]:
tempdf['CustLocation'] = tempdf['city']
tempdf.drop('city', axis=1, inplace=True)

In [41]:
tempdf

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),Age,age_group
0,T1,C5841053,1994-01-10,F,JAMSHEDPUR,17819.05,2016-08-02,14:32:07,25.0,22,20–25
1,T2,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-08-02,14:18:58,27999.0,59,46–60
2,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-08-02,14:27:12,459.0,19,<20
3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-08-02,14:27:14,2060.0,43,36–45
4,T5,C9031234,1988-03-24,F,NAVI MUMBAI,6714.43,2016-08-02,18:11:56,1762.5,28,26–35
...,...,...,...,...,...,...,...,...,...,...,...
1048562,T1048563,C8020229,1990-04-08,M,NEW DELHI,7635.19,2016-09-18,18:48:24,799.0,26,26–35
1048563,T1048564,C6459278,1992-02-20,M,NASHIK,27311.42,2016-09-18,18:37:34,460.0,24,20–25
1048564,T1048565,C6412354,1989-05-18,M,HYDERABAD,221757.06,2016-09-18,18:33:13,770.0,27,26–35
1048565,T1048566,C6420483,1978-08-30,M,PATNA,10117.87,2016-09-18,18:47:06,1000.0,38,36–45


In [44]:
# For saving as a .parquet file for preserving data types while reusing
# !pip install pyarrow if not already installed
import os

directory = 'data'

if not os.path.exists(directory):
    os.makedirs(directory)
    print(f"Directory '{directory}' created.")

file_path = os.path.join(directory, 'Transactions_Cleaned.parquet')
tempdf.to_parquet(file_path, index=False)
print(f"DataFrame successfully saved to '{file_path}'")

Directory 'data' created.
DataFrame successfully saved to 'data/Transactions_Cleaned.parquet'
